In [1]:
# Primer paso COCO_TO_DATAFRAME

In [12]:
from pylabel import importer
import numpy as np
import shutil
import yaml
import os

In [13]:
import json

In [42]:
f = open('/home/jupyter/11_categorias/train/labels.json')
data = json.load(f)

In [53]:
dict_class = []
for i in data['categories']:
    dict_class.append((i ['name'], i['id']))

In [55]:
a = dict(dict_class)

In [3]:
def coco_to_df(path_json, path_photos):
    
    dataset = importer.ImportCoco(path_json,
                                  path_to_images= path_photos,
                                  name="BCCD_coco")
    
    return dataset.df.reset_index()

In [4]:
# calcular el centro

def pascal_voc_to_yolo(x1, y1, x2, y2, image_w, image_h):
    return [((x2 + x1)/(2*image_w)), ((y2 + y1)/(2*image_h)), (x2 - x1)/image_w, (y2 - y1)/image_h]


In [5]:
def normalitation(df):
    
    df['yolo_metrics'] = df.apply(lambda row: pascal_voc_to_yolo(row.ann_bbox_xmin,
                                         row.ann_bbox_ymin,
                                         row.ann_bbox_xmax,
                                         row.ann_bbox_ymax,
                                         row.img_width,
                                         row.img_height),
                              axis=1)
    
    df['x_center'] = df['yolo_metrics'].apply(lambda x: x[0])
    df['y_center'] = df['yolo_metrics'].apply(lambda x: x[1])
    df['width'] = df['yolo_metrics'].apply(lambda x: x[2])
    df['height'] = df['yolo_metrics'].apply(lambda x: x[3])
    
    df['image_name'] =  df.img_filename
    df = df.rename(columns={"cat_name": "class_name",})
    df = df[['image_name', 'x_center', 'y_center', 'width', 'height', 'class_name']]
    
    return df

In [6]:
def set_categories(json_train_path):

    f = open(json_train_path)
    data = json.load(f)
    
    dict_categorias_map = []
    dict_categorias_final = []
    for i in data['categories']:
        dict_categorias_map.append((i ['name'], i['id']))
        dict_categorias_final.append((i['id'], i ['name']))

    
    return dict(dict_categorias_map), dict(dict_categorias_final)

In [7]:
def exist_folder(path):
    
    isExist = os.path.exists(path)
    if not isExist:
        os.makedirs(path, exist_ok=True)
        print("The new directory is created!")

In [8]:
def save_text_files(df, conjunto, variable):
    
    for i in list(set(df.image_name)):
        name = i.split('.')[0]
        data = df[df.image_name == i][['class_name', 'x_center', 'y_center', 'width', 'height']]
        np.savetxt(f'/home/jupyter/{variable}/coco128/labels/{conjunto}/{name}.txt', data.values, fmt='%s')
    

In [9]:
def copy_directories(src, dst):
    
    try:
        #if path already exists, remove it before copying with copytree()
        if os.path.exists(dst):
            shutil.rmtree(dst)
            shutil.copytree(src, dst)
    except OSError as e:
        # If the error was caused because the source wasn't a directory
        if e.errno == errno.ENOTDIR:
            shutil.copy(source_dir_prompt, destination_dir_prompt)
        else:
            print('Directory not copied. Error: %s' % e)

In [61]:
variable = '11_categorias'
df = coco_to_df(f'/home/jupyter/{variable}/train/labels.json', f'/home/jupyter/{variable}/train/data')
df_test = coco_to_df(f'/home/jupyter/{variable}/test/labels.json', f'/home/jupyter/{variable}/test/data')


df = normalitation(df)
df_test = normalitation(df_test)

dict_categorias, dict_categorias_final = set_categories(f'/home/jupyter/{variable}/train/labels.json')

df['class_name'] = df.class_name.map(dict_categorias)
df_test['class_name'] = df_test.class_name.map(dict_categorias)

In [62]:
df_test

,image_name,x_center,y_center,width,height,class_name
0,2741a1cc1eae6656.jpg,0.259375,0.569792,0.515625,0.593750,6
1,2741a1cc1eae6656.jpg,0.638281,0.543750,0.648438,0.570833,6
2,ee02895ca02e4515.jpg,0.435018,0.326715,0.870036,0.505415,6
3,ee02895ca02e4515.jpg,0.901625,0.351986,0.196751,0.303249,6
4,8ad516e98a8da3bc.jpg,0.494656,0.522436,0.796947,0.861111,6
...,...,...,...,...,...,...
6715,7bf9dd642155cf41.jpg,0.513255,0.504425,0.746686,0.827434,90
6716,7bf9dd642155cf41.jpg,0.180412,0.198009,0.057437,0.046460,90
6717,7bf9dd642155cf41.jpg,0.940353,0.268805,0.119293,0.192478,90
6718,d7dc0302cfa47b39.jpg,0.498437,0.439583,0.996875,0.879167,90


In [63]:
df

,image_name,x_center,y_center,width,height,class_name
0,aa03074bf3d68bb1.jpg,0.264062,0.597674,0.453125,0.667442,6
1,aa03074bf3d68bb1.jpg,0.597187,0.512210,0.754375,0.736047,6
2,cfb416e69ea6c6fa.jpg,0.487812,0.530417,0.808125,0.660833,6
3,190edecc570168f5.jpg,0.536875,0.482916,0.155000,0.400833,6
4,91173b5d90e594b4.jpg,0.514844,0.492188,0.967188,0.778125,6
...,...,...,...,...,...,...
16331,de59636068f7fa95.jpg,0.235156,0.422363,0.095313,0.100586,90
16332,de59636068f7fa95.jpg,0.285547,0.402343,0.077344,0.060547,90
16333,4014af073467f308.jpg,0.516667,0.582497,0.300000,0.600892,73
16334,4014af073467f308.jpg,0.675925,0.759754,0.034815,0.128205,90


In [10]:
def clean_data(variable):
        
    path = f'/home/jupyter/{variable}/coco128'
    exist_folder(path)
    
    path_labels = f'/home/jupyter/{variable}/coco128/labels'
    exist_folder(path_labels)
    
    path_images = f'/home/jupyter/{variable}/coco128/images'
    exist_folder(path_images)
    
    path_images_train = f'/home/jupyter/{variable}/coco128/images/train'
    exist_folder(path_images_train)
    
    path_images_test = f'/home/jupyter/{variable}/coco128/images/test'
    exist_folder(path_images_test)
    
    path_train = f'/home/jupyter/{variable}/coco128/labels/train'
    exist_folder(path_train)
    
    path_test = f'/home/jupyter/{variable}/coco128/labels/test'
    exist_folder(path_test)
    

    
    source_dir_train = f'/home/jupyter/{variable}/train/data'
    destination_dir_train = f'/home/jupyter/{variable}/coco128/images/train'
    copy_directories(source_dir_train, destination_dir_train)
    
    source_dir_test = f'/home/jupyter/{variable}/test/data'
    destination_dir_test = f'/home/jupyter/{variable}/coco128/images/test'
    copy_directories(source_dir_test, destination_dir_test)
    
    
    df = coco_to_df(f'/home/jupyter/{variable}/train/labels.json', f'/home/jupyter/{variable}/train/data')
    df_test = coco_to_df(f'/home/jupyter/{variable}/test/labels.json', f'/home/jupyter/{variable}/test/data')
    
    
    df = normalitation(df)
    df_test = normalitation(df_test)
    
    #categories = (set(df.class_name))
    dict_categorias, dict_categorias_final = set_categories(f'/home/jupyter/{variable}/train/labels.json')
    
    df['class_name'] = df.class_name.map(dict_categorias)
    df_test['class_name'] = df_test.class_name.map(dict_categorias)
    

    save_text_files(df, 'train', variable)
    save_text_files(df_test, 'test', variable)
    
    dict_final = {'path': f'/home/jupyter/{variable}/coco128', 'train': "images/train" ,'val': 'images/test', 'names': dict_categorias_final}
    
    
    with open(f'/home/jupyter/{variable}/YOLO_store.yaml', 'w') as file:
        documents = yaml.dump(dict_final, file)
    
    return dict_final

In [14]:
clean_data('11_categorias')

{'path': '/home/jupyter/11_categorias/coco128',
 'train': 'images/train',
 'val': 'images/test',
 'names': {0: 'Accordion',
  1: 'Adhesive tape',
  2: 'Aircraft',
  3: 'Airplane',
  4: 'Alarm clock',
  5: 'Alpaca',
  6: 'Ambulance',
  7: 'Animal',
  8: 'Ant',
  9: 'Antelope',
  10: 'Apple',
  11: 'Armadillo',
  12: 'Artichoke',
  13: 'Auto part',
  14: 'Axe',
  15: 'Backpack',
  16: 'Bagel',
  17: 'Baked goods',
  18: 'Balance beam',
  19: 'Ball',
  20: 'Balloon',
  21: 'Banana',
  22: 'Band-aid',
  23: 'Banjo',
  24: 'Barge',
  25: 'Barrel',
  26: 'Baseball bat',
  27: 'Baseball glove',
  28: 'Bat (Animal)',
  29: 'Bathroom accessory',
  30: 'Bathroom cabinet',
  31: 'Bathtub',
  32: 'Beaker',
  33: 'Bear',
  34: 'Bed',
  35: 'Bee',
  36: 'Beehive',
  37: 'Beer',
  38: 'Beetle',
  39: 'Bell pepper',
  40: 'Belt',
  41: 'Bench',
  42: 'Bicycle',
  43: 'Bicycle helmet',
  44: 'Bicycle wheel',
  45: 'Bidet',
  46: 'Billboard',
  47: 'Billiard table',
  48: 'Binoculars',
  49: 'Bird',
  5